In [1]:
import pandas as pd

# Đọc dữ liệu
df_credit = pd.read_csv('./credit_record.csv')
df_application = pd.read_csv('./application_record.csv')

# Loại bỏ các giá trị ID trùng nhau của df_application
df_application=df_application.drop_duplicates(subset=df_application.columns[1:], keep='first')

# Thay thế các giá trị bị trống
df_application['OCCUPATION_TYPE'].fillna(value='Other', inplace=True)

# Bỏ cột FLAG_MOBIL vì nó chỉ có 1 giá trị
df_application.drop('FLAG_MOBIL', axis=1, inplace=True)

# Loại bỏ outliner cho thuộc tính [CNT_CHILDREN]
df_application = df_application[df_application['CNT_CHILDREN'] <= 5]

# Tạo thuộc tính target để xác định khách hàng có nguy cơ cao nợ xấu
df_credit['target'] = df_credit['STATUS']
df_credit['target'].replace('X', 0, inplace=True)
df_credit['target'].replace('C', 0, inplace=True)
df_credit['target'] = df_credit['target'].astype(int)
df_credit.loc[df_credit['target']>=1,'target'] = 1

# target=1 (nguy cơ cao) nếu có ít nhất một tháng mà khách hàng trễ thanh toán từ 30 ngày trở lên
df_target = pd.DataFrame(df_credit.groupby(['ID'])['target'].agg(max)).reset_index()

# Kết hợp dataframe
df = pd.merge(df_application, df_target, how='inner', on=['ID'])

# Tài khoản đã được mở trong bao nhiêu tháng
df_start = pd.DataFrame(df_credit.groupby(['ID'])['MONTHS_BALANCE'].agg(min)).reset_index()

# Đổi tên thuộc tính
df_start.rename(columns={'MONTHS_BALANCE':'ACCOUNT_LENGTH'}, inplace=True)

# Đổi kiểu dữ liệu từ âm sang dương
df_start['ACCOUNT_LENGTH'] = -df_start['ACCOUNT_LENGTH']

# Kết hợp dataframe
df = pd.merge(df, df_start, how='inner', on=['ID'])

# Tạo thuộc tính age để tính tuổi
df['AGE_YEARS'] = (-df['DAYS_BIRTH'] / 365).astype(int)
df.drop('DAYS_BIRTH', axis=1, inplace=True)

# tạo thuộc tính unemployed để xác định ai đang thất nghiệp
df['UNEMPLOYED'] = 0
df.loc[-df['DAYS_EMPLOYED']<0,'UNEMPLOYED'] = 1

# Tạo thuôc tính year_employed để xác định số năm đang đi làm
df['YEARS_EMPLOYED'] = (-df['DAYS_EMPLOYED']/365).astype(int)
df.loc[df['YEARS_EMPLOYED']<0,'YEARS_EMPLOYED'] = 0
df.drop('DAYS_EMPLOYED', axis=1, inplace=True)

# Đổi kiểu dữ liệu thành binary
df["FLAG_OWN_CAR"] = df["FLAG_OWN_CAR"].replace(["Y","N"],[1,0])
df["FLAG_OWN_REALTY"] = df["FLAG_OWN_REALTY"].replace(["Y","N"],[1,0])

# Đổi tên thuộc tính
df = df.rename(columns={'CODE_GENDER': 'Gender', 'FLAG_OWN_CAR': 'OwnCar', 
                               'FLAG_OWN_REALTY':'OwnProperty','CNT_CHILDREN':'NumChildren',
                               'AMT_INCOME_TOTAL':'TotalIncome','NAME_INCOME_TYPE':'IncomeType',
                               'NAME_EDUCATION_TYPE':'EducationType','NAME_FAMILY_STATUS':'FamilyStatus',
                               'NAME_HOUSING_TYPE':'HousingType','FLAG_WORK_PHONE':'WorkPhone',
                               'FLAG_PHONE':'Phone','FLAG_EMAIL':'Email','OCCUPATION_TYPE':'OccupationType',
                               'CNT_FAM_MEMBERS':'NumFamily','target':'Target','ACCOUNT_LENGTH':'AccountLength',
                               'AGE_YEARS':'Age','UNEMPLOYED':'Unemployed','YEARS_EMPLOYED':'YearsEmployed'})

# Chuyển kiểu dữ liệu cho thuộc tính NumFamily
df['NumFamily'] = df['NumFamily'].astype(int)

# Sắp xếp lại vị trí các thuộc tính
df = df[['ID', 'Gender', 'OwnCar', 'OwnProperty', 'WorkPhone',
               'Phone', 'Email', 'Unemployed', 'NumChildren', 'NumFamily', 
               'AccountLength', 'TotalIncome', 'Age', 'YearsEmployed',  
               'IncomeType', 'EducationType', 'FamilyStatus',
               'HousingType', 'OccupationType','Target']]

# Xuất ra file data.csv
df.to_csv('data.csv',index=False)

